# Using SimStore and the OpenPathSampling CLI

This tutorial covers both the new storage subsystem, SimStore, and the OpenPathSampling Command Line Interface (CLI). It also shows how to use test systems from OpenMMTools as toy examples. This can be particularly useful for method development, as the OpenMMTools test systems add a step up in complexity from the OPS internal toy engine.

The OpenPathSampling Command Line Interface (CLI) makes it easier to run your OPS simulations, especially in cluster environments. The basic approach is to first create the simulation objects, including initial conditions, and to save those to a file. Think of this file as a database of simulation setup information that you can later load with the CLI

In [ ]:
import openpathsampling as paths
import openmmtools
from simtk import unit
import numpy as np
import mdtraj as md

In [ ]:
# to use SimStore, we need to monkey patch and import specific CVs, Storage
from openpathsampling.experimental.storage import monkey_patch_all, Storage
from openpathsampling.experimental.storage.collective_variables import MDTrajFunctionCV

paths = monkey_patch_all(paths)
paths.InterfaceSet.simstore = True

## TIS Setup

Here we'll set up a double-well dimer in a bath of WCA particles. This is a very common toy system for rare events, and is included in OpenMMTools.

All particles in this system have a WCA nonbonded interaction, but two of them have a quartic double well "bond", which allows them to either be in a "condensed" (short) state or an "extended" (long) state. The quartic potential is defined by:

$$
V_{dw}(r) = h \left(1 - \left(\frac{r - r_0 - w}{w}\right)^2\right)^2
$$

where $r$ is the distance between the particles, and the parameters $r_0$, $w$, and $h$ define the minima and maximum of the quartic potential such that the two wells have minima at $(r_0, 0)$ and $(r_0 + 2w, 0)$, and the barrier has a maximum at $(r_0 + w, h)$. The potential (with the interface locations shown in grey) looks like this:

![Double well PES](double_well.png)

### Create "dimensionless" units

In [ ]:
# energy in OpenMM is energy/mol, so k in kT is k_B*N_A
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

In [ ]:
# argon-like WCA fluid parameters
# these are units of mass, energy, and distance
mass = 39.9 * unit.dalton
epsilon = 120. * unit.kelvin * kB
sigma = 3.4 * unit.angstroms

In [ ]:
# tau is the "unitless" time unit
tau = np.sqrt(sigma**2 * mass / epsilon)

In [ ]:
# this is so we can use MDTraj more easily
sigma_nm = sigma.value_in_unit(unit.nanometer)
print(sigma_nm)

### Set some parameters we'll use

In [ ]:
temperature = 0.824 / kB * epsilon

In [ ]:
# double-well parameters based on van Erp et al. JCP 2003
h = 6.0 * kB * temperature
r0 = 2.**(1. / 6.) * sigma
w = 0.25 * sigma

### Engine Setup

In [ ]:
# 1. Set up the OpenMM side

# these are all the default parameters, but we'll make it explicit
testsystem = openmmtools.testsystems.DoubleWellDimer_WCAFluid(
    ndimers=1, 
    nparticles=216,
    density=0.96,
    mass=mass,
    epsilon=epsilon,
    sigma=sigma,
    h=h,
    r0=r0,
    w=w
)

integrator = openmmtools.integrators.VVVRIntegrator(
    timestep=0.001 * tau,
    collision_rate=2.0 / tau,
    temperature=temperature
)

In [ ]:
# 2. Use the OPS wrappers for OpenMM

topology = paths.engines.MDTrajTopology(testsystem.mdtraj_topology)
engine = paths.engines.openmm.Engine(
    topology=topology,
    system=testsystem.system,
    integrator=integrator,
    options={'n_frames_max': 2000,
             'n_steps_per_frame': 10}
).named('engine')

In [ ]:
# 3. Get an initial snapshot

# use OpenMM simulation object to set information
engine.simulation.context.setPositions(testsystem.positions)
engine.simulation.minimizeEnergy()
snap = engine.current_snapshot

### Define CVs, stable states, and interfaces

In [ ]:
# OLD:
#cv = paths.MDTrajFunctionCV("r", md.compute_distances, topology=topology,
#                            atom_pairs=[[0,1]])
cv = MDTrajFunctionCV(
    md.compute_distances,
    topology=topology,
    atom_pairs=[[0,1]]
).named("r")

In [ ]:
# stable states
condensed = paths.CVDefinedVolume(
    cv,
    lambda_min=float("-inf"),
    lambda_max=1.20 * sigma_nm
).named("condensed")

extended = paths.CVDefinedVolume(
    cv,
    lambda_min=1.54 * sigma_nm,
    lambda_max=float("inf")
).named("extended")

In [ ]:
# TIS interfaces
interfaces = paths.VolumeInterfaceSet(
    cv,
    minvals=float("-inf"),
    maxvals=np.array([1.20, 1.26, 1.32]) * sigma_nm
).named("interfaces")

### Create network and move scheme

In [ ]:
network = paths.MISTISNetwork([(condensed, interfaces, extended)]).named("tis")
scheme = paths.DefaultScheme(network, engine).named("retis_scheme")

## Save everything

In [ ]:
# OLD
#storage = paths.Storage("setup.nc", mode='w')
#storage.save(paths.Trajectory([snap]))  # save as a trajectory for templating

storage = Storage("setup.db", mode='w')
storage.save(snap)  # now because we need initial conditions, not a template!
storage.save(scheme)  # scheme contains all the simulation info!

In [ ]:
# we may find tau and sigma_nm useful in analysis, so we store them, too
storage.tags['tau'] = tau
storage.tags['sigma_nm'] = sigma_nm

storage.close()

## High-Temperature MD Setup

To obtain an initial trajectory (which we will then need to equilibrate), we will use high temperature dynamics for this example. High temperature dynamics is a reasonable way to get an initial trajectory, but depending on your system and what information you have about it, there may be better ways.

We'll create the high-temperature engine and save it by appending to the `setup.db` file.

In [ ]:
hi_temp = openmmtools.integrators.VVVRIntegrator(
    timestep=0.001 * tau,
    collision_rate=2.0 / tau,
    temperature=2 * temperature
)

hi_temp_engine = paths.engines.openmm.Engine(
    topology=topology,
    system=testsystem.system,
    integrator=hi_temp,
    options={'n_frames_max': 10000,
             'n_steps_per_frame': 10}
).named('hi_temp')

In [ ]:
storage = Storage("setup.db", mode='a')
storage.save(hi_temp_engine)
storage.close()